In [1]:
import pandas

In [42]:
# %load hello.py
import pandas

if __name__ == "__main__":
	data = pandas.read_csv("MODIS_C6_USA_contiguous_and_Hawaii_7d.csv")
    
print(data)
data['latlongtuple'] = list(zip(data.latitude, data.longitude))

df = pandas.DataFrame(data)
df.to_csv(r'output.csv', index = False)
# pandas.DataFrame.to_csv(data)

      latitude  longitude  brightness  scan  track    acq_date  acq_time  \
0       19.226    -71.786       300.8   1.4    1.2  2020-03-21       320   
1       19.268    -71.874       300.8   1.4    1.2  2020-03-21       320   
2       19.274    -71.882       301.4   1.4    1.2  2020-03-21       320   
3       20.003    -75.678       340.7   1.0    1.0  2020-03-21       320   
4       20.001    -75.688       330.4   1.0    1.0  2020-03-21       320   
...        ...        ...         ...   ...    ...         ...       ...   
4919    17.950    -98.563       329.6   1.0    1.0  2020-03-28      1715   
4920    18.173   -100.205       330.0   1.0    1.0  2020-03-28      1715   
4921    18.161   -100.247       336.1   1.0    1.0  2020-03-28      1715   
4922    17.612    -98.769       331.7   1.0    1.0  2020-03-28      1715   
4923    17.610    -98.759       322.7   1.0    1.0  2020-03-28      1715   

     satellite  confidence version  bright_t31   frp daynight  
0            T         

In [8]:
import pandas as pd
import numpy as np
import math

import geopandas
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

In [43]:
# neighborhood_data = pd.read_csv('https://raw.githubusercontent.com/JimKing100/SF_Real_Estate_Live/master/data/neighborhood_data.csv')


In [27]:
# Read the geojson map file for Realtor Neighborhoods into a GeoDataframe object
sf = geopandas.read_file('https://raw.githubusercontent.com/JimKing100/SF_Real_Estate_Live/master/data/Realtor%20Neighborhoods.geojson')

# Set the Coordinate Referance System (crs) for projections
# ESPG code 4326 is also referred to as WGS84 lat-long projection
sf.crs = {'init': 'epsg:4326'}

# Rename columns in geojson map file
sf = sf.rename(columns={'geometry': 'geometry','nbrhood':'neighborhood_name', 'nid': 'subdist_no'}).set_geometry('geometry')

# Change neighborhood id (subdist_no) for correct code for Mount Davidson Manor and for parks
sf.loc[sf['neighborhood_name'] == 'Mount Davidson Manor', 'subdist_no'] = '4n'
sf.loc[sf['neighborhood_name'] == 'Golden Gate Park', 'subdist_no'] = '12a'
sf.loc[sf['neighborhood_name'] == 'Presidio', 'subdist_no'] = '12b'
sf.loc[sf['neighborhood_name'] == 'Lincoln Park', 'subdist_no'] = '12c'

sf.sort_values(by=['subdist_no'])

,neighborhood_name,subdist_no,sfar_distr,geometry
3,Bayview,10a,District 10 - Southeast,"MULTIPOLYGON (((-122.38759 37.75026, -122.3874..."
11,Crocker Amazon,10b,District 10 - Southeast,"MULTIPOLYGON (((-122.42470 37.71022, -122.4245..."
16,Excelsior,10c,District 10 - Southeast,"MULTIPOLYGON (((-122.42558 37.73148, -122.4239..."
57,Outer Mission,10d,District 10 - Southeast,"MULTIPOLYGON (((-122.44562 37.71197, -122.4489..."
79,Visitacion Valley,10e,District 10 - Southeast,"MULTIPOLYGON (((-122.42335 37.70925, -122.4241..."
...,...,...,...,...
66,Potrero Hill,9e,District 9 - Central East,"MULTIPOLYGON (((-122.39434 37.76658, -122.3935..."
73,South of Market,9f,District 9 - Central East,"MULTIPOLYGON (((-122.40118 37.77834, -122.4009..."
18,Yerba Buena,9g,District 9 - Central East,"MULTIPOLYGON (((-122.39430 37.78378, -122.3942..."
46,South Beach,9h,District 9 - Central East,"MULTIPOLYGON (((-122.39120 37.79578, -122.3924..."


In [26]:
def json_data(selectedYear):
    yr = selectedYear
    
    # Pull selected year from neighborhood summary data
    df_yr = neighborhood_data[neighborhood_data['year'] == yr]
    
    # Merge the GeoDataframe object (sf) with the neighborhood summary data (neighborhood)
    merged = pd.merge(sf, df_yr, on='subdist_no', how='left')
    
    # Fill the null values
    values = {'year': yr, 'sale_price_count': 0, 'sale_price_mean': 0, 'sale_price_median': 0,
              'sf_mean': 0, 'price_sf_mean': 0, 'min_income': 0}
    merged = merged.fillna(value=values)
    
    # Bokeh uses geojson formatting, representing geographical features, with json
    # Convert to json
    merged_json = json.loads(merged.to_json())
    
    # Convert to json preferred string-like object 
    json_data = json.dumps(merged_json)
    return json_data

# This dictionary contains the formatting for the data in the plots
format_data = [('sale_price_count', 0, 100,'0,0', 'Number of Sales'),
               ('sale_price_mean', 500000, 4000000,'$0,0', 'Average Sales Price'),
               ('sale_price_median', 500000, 4000000, '$0,0', 'Median Sales Price'),
               ('sf_mean', 500, 5000,'0,0', 'Average Square Footage'),
               ('price_sf_mean', 0, 2000,'$0,0', 'Average Price Per Square Foot'),
               ('min_income', 50000, 600000,'$0,0', 'Minimum Income Required')
              ]
 
#Create a DataFrame object from the dictionary 
format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

In [ ]:
# Create a plotting function
def make_plot(field_name):    
  # Set the format of the colorbar
  min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
  max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
  field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]

  # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
  color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

  # Create color bar.
  format_tick = NumeralTickFormatter(format=field_format)
  color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, formatter=format_tick,
  border_line_color=None, location = (0, 0))

  # Create figure object.
  verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]

  p = figure(title = verbage + ' by Neighborhood for Single Family Homes in SF by Year - 2009 to 2018', 
             plot_height = 650, plot_width = 850,
             toolbar_location = None)
  p.xgrid.grid_line_color = None
  p.ygrid.grid_line_color = None
  p.axis.visible = False

  # Add patch renderer to figure. 
  p.patches('xs','ys', source = geosource, fill_color = {'field' : field_name, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
  
  # Specify color bar layout.
  p.add_layout(color_bar, 'right')

  # Add the hover tool to the graph
  p.add_tools(hover)
  return p

In [ ]:
# Define the callback function: update_plot
def update_plot(attr, old, new):
    # The input yr is the year selected from the slider
    yr = slider.value
    new_data = json_data(yr)
    
    # The input cr is the criteria selected from the select box
    cr = select.value
    input_field = format_df.loc[format_df['verbage'] == cr, 'field'].iloc[0]
    
    # Update the plot based on the changed inputs
    p = make_plot(input_field)
    
    # Update the layout, clear the old document and display the new document
    layout = column(p, widgetbox(select), widgetbox(slider))
    curdoc().clear()
    curdoc().add_root(layout)
    
    # Update the data
    geosource.geojson = new_data

In [25]:
# Input geojson source that contains features for plotting for:
# initial year 2018 and initial criteria sale_price_median
geosource = GeoJSONDataSource(geojson = json_data(2018))
input_field = 'sale_price_median'

# Define a sequential multi-hue color palette.
palette = brewer['Blues'][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Add hover tool
hover = HoverTool(tooltips = [ ('Neighborhood','@neighborhood_name'),
                               ('# Sales', '@sale_price_count'),
                               ('Average Price', '$@sale_price_mean{,}'),
                               ('Median Price', '$@sale_price_median{,}'),
                               ('Average SF', '@sf_mean{,}'),
                               ('Price/SF ', '$@price_sf_mean{,}'),
                               ('Income Needed', '$@min_income{,}')])

# Call the plotting function
p = make_plot(input_field)

# Make a slider object: slider 
slider = Slider(title = 'Year',start = 2009, end = 2018, step = 1, value = 2018)
slider.on_change('value', update_plot)

# Make a selection object: select
select = Select(title='Select Criteria:', value='Median Sales Price', options=['Median Sales Price', 'Minimum Income Required',
                                                                               'Average Sales Price', 'Average Price Per Square Foot',
                                                                               'Average Square Footage', 'Number of Sales'])
select.on_change('value', update_plot)

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
# Display the current document
layout = column(p, widgetbox(select), widgetbox(slider))
curdoc().add_root(layout)

In [24]:
# # This dictionary contains the formatting for the data in the plots
# format_data = [('sale_price_count', 0, 100,'0,0', 'Number of Sales'),
#                ('sale_price_mean', 500000, 4000000,'$0,0', 'Average Sales Price'),
#                ('sale_price_median', 500000, 4000000, '$0,0', 'Median Sales Price'),
#                ('sf_mean', 500, 5000,'0,0', 'Average Square Footage'),
#                ('price_sf_mean', 0, 2000,'$0,0', 'Average Price Per Square Foot'),
#                ('min_income', 50000, 600000,'$0,0', 'Minimum Income Required')
#               ]
 
# #Create a DataFrame object from the dictionary 
# format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

In [23]:
# Add hover tool
hover = HoverTool(tooltips = [ ('Neighborhood','@neighborhood_name'),
                               ('# Sales', '@sale_price_count'),
                               ('Average Price', '$@sale_price_mean{,}'),
                               ('Median Price', '$@sale_price_median{,}'),
                               ('Average SF', '@sf_mean{,}'),
                               ('Price/SF ', '$@price_sf_mean{,}'),
                               ('Income Needed', '$@min_income{,}')])


# Make a slider object: slider 
slider = Slider(title = 'Year',start = 2009, end = 2018, step = 1, value = 2018)
slider.on_change('value', update_plot)

# Make a selection object: select
select = Select(title='Select Criteria:', value='Median Sales Price', options=['Median Sales Price', 'Minimum Income Required',
                                                                               'Average Sales Price', 'Average Price Per Square Foot',
                                                                               'Average Square Footage', 'Number of Sales'])
select.on_change('value', update_plot)